## LangChain 프롬프트 구성

### OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
import gradio as gr

# .env 로드 및 API 키 확인
load_dotenv()
print(f"{os.getenv('OPENAI_API_KEY')[:9]}***")  # 따옴표 중첩 문제 해결

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
print(f"LLM 모델: {llm.model_name}")

### 시스템 역할 메시지 구성

In [ ]:
# 초기 시스템 메시지
system_message = SystemMessage(
    content="당신은 '명탐정 코난' 전문가입니다. 질문에 친절하고 정확하게 답해 주세요."
)
initial_messages = [system_message]

### 채팅 처리 함수 (`add_messages` 사용)

1. `add_messages(left, right)` (리스트 + 리스트)
    - `left` 리스트의 메시지들을 우선 저장한 뒤, `right` 리스트의 메시지들로 덮어쓰기 또는 추가를 수행합니다.
    - ID가 중복된 메시지는 `right`의 메시지로 교체(덮어쓰기)되고, 새로운 ID는 리스트 끝에 순서대로 추가됩니다.

2. `add_messages(state["messages"], new_message)` (리스트 + 단일 메시지)
    - 내부적으로 단일 메시지를 리스트로 감싸 동일한 로직을 적용합니다.
    - 결과적으로 기존 메시지 뒤에 새로운 메시지가 추가됩니다.

In [ ]:
# 채팅 처리 함수
def chat_fn(user_input, chat_history):
    if chat_history is None:
        chat_history = initial_messages.copy()

    # 사용자 메시지 추가
    human_msg = HumanMessage(content=user_input)
    chat_history = add_messages(chat_history, [human_msg])

    # LLM 응답 생성 및 추가
    ai_msg = llm.invoke(chat_history)
    chat_history = add_messages(chat_history, [ai_msg])

    # OpenAI 스타일 메시지로 변환 (type="messages" 대응)
    chat_display = []
    for msg in chat_history:
        if isinstance(msg, HumanMessage):
            chat_display.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            chat_display.append({"role": "assistant", "content": msg.content})

    return chat_display, chat_history

### Gradio 인터페이스 구성
패키지 설치 `pip install gradio`

In [ ]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️‍♂️ 명탐정 코난 전문가에게 무엇이든 물어보세요!")

    chatbot = gr.Chatbot(height=300, type="messages", label="명탐정 코난 전문가 챗봇")
    txt = gr.Textbox(placeholder="질문을 입력하세요...", show_label=False)
    state = gr.State(initial_messages.copy())

    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)  # 입력창 초기화

demo.launch()

-----
** End of Documents **